# CATÁLOGO DE DADOS


### VISÃO GERAL DO CATÁLOGO
**Nome do conjunto de dados:** 
  Despesas pela Cota para Exercício da Atividade Parlamentar (CEAP)

**Fontes oficiais:**
- [Portal de Dados Abertos da Câmara dos Deputados](https://dadosabertos.camara.leg.br)
- [Arquivos Disponíveis](https://dadosabertos.camara.leg.br/swagger/api.html?tab=staticfile)
- [Documentação dos campos](https://dadosabertos.camara.leg.br/howtouse/2023-12-26-dados-ceap.html)
- [Dataset Utilizado](https://www.camara.leg.br/cotas/Ano-2024.csv.zip)

  
**Frequência de atualização:**
  Publicação mensal, com revisões eventuais

### LINHAGEM DOS DADOS
O dataset de dados utilizado neste trabalho foi obtidos a partir da base pública da Câmara dos Deputados, disponibilizada em diversos formatos.

Os dados foram coletados e baixados no formato CSV.ZIP, no qual foram extraídos e armazenados tanto o arquivo original .zip, quanto o extraído .csv no volume do SCHEMA staging, preservando a estrutura original da fonte.

A **camada Bronze** irá preservar a estrutura original da fonte, sem alterações de schema, garantindo reprodutibilidade e rastreabilidade.

Na **camada Silver**, os dados passaram por processos de limpeza, padronização de tipos (datas, valores numéricos e campos categóricos), tratamento de valores nulos e remoção de registros inconsistentes.

Na **camada Gold**, os dados foram reorganizados em um modelo analítico no formato Esquema Estrela, composto por uma tabela fato e tabelas dimensão, visando facilitar análises e consultas analíticas.



## CATÁLOGO — TABELA FATO

**Tabela:** FATO_DESPESA_CEAP

**Descrição:** Tabela central do modelo analítico que registra cada despesa individual declarada por um parlamentar no âmbito da CEAP.

**Granularidade:** Uma linha representa uma despesa individual associada a um documento fiscal declarado por um parlamentar.

**Campos da Tabela Fato**
| Campo           | Tipo          | Descrição                                   | Domínio / Regras      |
| --------------- | ------------- | ------------------------------------------- | --------------------- |
| id_documento    | String        | Identificador único do documento da despesa | Não nulo              |
| valor_despesa   | Decimal(12,2) | Valor monetário da despesa declarada        | ≥ 0                   |
| data_despesa    | Date          | Data de emissão do documento                | yyyy-mm-dd            |
| id_parlamentar  | Integer       | Identificador do parlamentar                | FK → DIM_PARLAMENTAR  |
| id_tempo        | Integer       | Chave da dimensão tempo                     | FK → DIM_TEMPO        |
| id_tipo_despesa | Integer       | Código do tipo de despesa                   | FK → DIM_TIPO_DESPESA |
| id_partido      | Integer       | Identificador do partido político           | FK → DIM_PARTIDO      |
| id_uf           | Integer       | Identificador da unidade federativa         | FK → DIM_UF           |

---


### CATÁLOGO — DIMENSÕES

#### DIM_PARLAMENTAR
**Descrição:** Dimensão que contém informações descritivas dos parlamentares que utilizaram a CEAP.
| Campo            | Tipo    | Descrição                          | Domínio          |
| ---------------- | ------- | ---------------------------------- | ---------------- |
| id_parlamentar   | Integer | Identificador único do parlamentar | Não nulo         |
| nome_parlamentar | String  | Nome do parlamentar                | Texto            |
| legislatura      | Integer | Número da legislatura              | Inteiro positivo |
| sigla_partido    | String  | Sigla do partido político          | Ex: PT, PL, PSDB |
| sigla_uf         | String  | Unidade federativa do parlamentar  | Ex: SP, MG       |

---
#### DIM_TEMPO
**Descrição:** Dimensão temporal utilizada para análises cronológicas e agregações por período.
| Campo     | Tipo    | Descrição                          | Domínio          |
| --------- | ------- | ---------------------------------- | ---------------- |
| id_tempo  | Integer | Chave substituta da dimensão tempo | Não nulo         |
| data      | Date    | Data completa                      | yyyy-mm-dd       |
| ano       | Integer | Ano da despesa                     | ≥ 1900           |
| mes       | Integer | Mês da despesa                     | 1 a 12           |
| trimestre | Integer | Trimestre do ano                   | 1 a 4            |
| nome_mes  | String  | Nome do mês                        | Janeiro–Dezembro |

---
#### DIM_TIPO_DESPESA
**Descrição:** Classifica o tipo de despesa conforme categorização oficial da CEAP.
| Campo                  | Tipo    | Descrição                 | Domínio       |
| ---------------------- | ------- | ------------------------- | ------------- |
| id_tipo_despesa        | Integer | Código do tipo de despesa | Conforme CEAP |
| descricao_tipo_despesa | String  | Descrição textual do tipo | Texto         |

---
#### DIM_PARTIDO
**Descrição:** Dimensão que representa os partidos políticos aos quais os parlamentares estão vinculados.
| Campo         | Tipo    | Descrição                   | Domínio    |
| ------------- | ------- | --------------------------- | ---------- |
| id_partido    | Integer | Chave substituta do partido | Não nulo   |
| sigla_partido | String  | Sigla do partido            | Ex: PT, PL |
| nome_partido  | String  | Nome completo do partido    | Texto      |

---
#### DIM_UF
**Descrição:** Dimensão geográfica baseada na unidade federativa do parlamentar.
| Campo    | Tipo    | Descrição                   | Domínio    |
| -------- | ------- | --------------------------- | ---------- |
| id_uf    | Integer | Chave substituta da UF      | Não nulo   |
| sigla_uf | String  | Sigla da unidade federativa | Ex: SP, RJ |
| nome_uf  | String  | Nome completo do estado     | Texto      |

## 📐 Diagrama de Modelagem – Esquema Estrela (CEAP)

erDiagram
    FATO_DESPESA_CEAP {
        string  id_documento
        int     id_parlamentar
        int     id_tempo
        int     id_tipo_despesa
        string  id_partido
        string  id_uf
        decimal valor_documento
        decimal valor_glosa
        decimal valor_liquido
    }

    DIM_PARLAMENTAR {
        int     id_parlamentar
        string  nome_parlamentar
        string  sigla_partido
        string  sigla_uf
        int     num_legislatura
    }

    DIM_TEMPO {
        int     id_tempo
        date    data
        int     ano
        int     mes
        int     trimestre
        string  nome_mes
    }

    DIM_TIPO_DESPESA {
        int     id_tipo_despesa
        string  descricao_tipo_despesa
        int     id_especificacao_tipo
        string  descricao_especificacao_tipo
    }

    DIM_PARTIDO {
        string  id_partido
        string  sigla_partido
    }

    DIM_UF {
        string  id_uf
        string  sigla_uf
    }

    DIM_PARLAMENTAR ||--o{ FATO_DESPESA_CEAP : id_parlamentar
    DIM_TEMPO ||--o{ FATO_DESPESA_CEAP : id_tempo
    DIM_TIPO_DESPESA ||--o{ FATO_DESPESA_CEAP : id_tipo_despesa
    DIM_PARTIDO ||--o{ FATO_DESPESA_CEAP : id_partido
    DIM_UF ||--o{ FATO_DESPESA_CEAP : id_uf
